# Combine and Clean Data Sources

This notbook is used to combine all data in to a single Sqlite database. The cleaning evolved heavily so at some point, all this cleaning should be refactored into a seperate module.

In [1]:
%load_ext lab_black

In [2]:
from pathlib import Path
import json
import gettext

from cleantext import clean
import dataset
import requests
import pycountry
from tqdm import tqdm
from datetime import date, datetime

In [3]:
german = gettext.translation("iso3166", pycountry.LOCALES_DIR, languages=["de"])
german.install()

## Manual Fixes

There are sometimes errors in the data. Since each incident has a uniquie id, fix it here.

In [4]:
manual_fixes = {
    "ca0dee4be1029c2ab24cdea755b88086": {"city": "Halle (Saale)"},
    "https://www.raa-sachsen.de/support/chronik/vorfaelle/bon-courage-fassungslos-ueber-naziuebergriffe-auf-dem-bornaer-stadtfest-3219": {
        "city": "Borna"
    },
    "mobile-beratung-a10d62daa23594df92c9704e1606dcfc": {
        "city": "Oranienbaum-Wörlitz",
        "county": "Wittenberg",
    },
    "mobile-beratung-000dad6aace24b3d8ebae940a8de8e72": {"city": "Dessau"},
    "https://www.raa-sachsen.de/support/chronik/vorfaelle/goerlitz-2779": {
        "date": date(2010, 11, 14)
    },
    "https://www.raa-sachsen.de/support/chronik/vorfaelle/leipzig-reudnitz-4976": {
        "date": date(2020, 11, 13)
    },
}

## Valid Regions

Taken regions.json from <https://github.com/datenguide/metadata>

These files contains a list of all valid regios. If a region is not in there, filter it out. This greatly the performance of the geocoding api. It's not optimal that we acutlaly throw this information await. This information should be kept actually. (TODO)

In [5]:
with open("regions.json") as json_file:
    regions = json.load(json_file)

regions = list(regions.values())
regions_counties = [x for x in regions if x["level"] == 3]


def is_valid_county(county):
    return (
        len(
            [
                x
                for x in regions_counties
                if x["name"].startswith(county)
                and x["duration"]["until"] == "2019-12-31T00:00:00.000Z"
            ]
        )
        != 0
    )

Read it some secrets later needed to comunicate with an internal API

In [6]:
auth = tuple(Path("secrets.txt").read_text().split()[1:])

In [7]:
all_incidents = []
all_src = []
all_chronicle = []

for p in Path("data").glob("*.db"):
    print(p)
    db = dataset.connect("sqlite:///" + str(p))
    all_incidents += db["incidents"].all()
    all_src += db["sources"].all()

    new_chro = list(db["chronicle"].all())
    if len(new_chro) == 0:
        new_chro = db["chronicles"].all()

    all_chronicle += new_chro

data/muenchen-chronik-scraper.db
data/mpower-scraper.db
data/ezra-converter.db
data/reachout-scraper.db
data/response-scraper.db
data/leuchtlinie-scraper.db
data/kleine-randnotiz-converter.db
data/mobile-opferberatung-scraper.db
data/raa-sachsen-scraper.db
data/opferperspektive-scraper.db
data/opferberatung-rheinland-scraper.db
data/lobbi-scraper.db


In [8]:
for x in all_chronicle:
    if "region" in x and len(x["region"]) > 0:
        continue
    if "iso3166_2" in x and x["iso3166_2"] != None and len(x["iso3166_2"]) > 0:
        x["region"] = pycountry.subdivisions.get(code=x["iso3166_2"]).name
    elif "iso3166_1" in x and x["iso3166_1"] != None and len(x["iso3166_1"]) > 0:
        x["region"] = pycountry.countries.get(alpha_2=x["iso3166_1"]).name
    else:
        raise ValueError("Need to specify region somehow")

In [9]:
# only using date (and without time (hour/minute)) for now


def ensure_date(x):
    if x is None:
        return x
    if isinstance(x, datetime):
        return x.date()
    elif isinstance(x, date):
        return x
    ValueError("neither date or datetime")


for row in all_src:
    if "date" in row:
        row["date"] = ensure_date(row["date"])
    else:
        row["date"] = None

In [10]:
db = dataset.connect("sqlite:///rechtegewalt.db")

In [11]:
tab_incidents = db["incidents"]
for x in all_incidents:
    if "id" in x:
        x.pop("id")
tab_incidents.insert_many(all_incidents)

tab_src = db["sources"]
for x in all_src:
    if "id" in x:
        x.pop("id")
tab_src.insert_many(all_src)

tab_chro = db["chronicles"]
for x in all_chronicle:
    if "id" in x:
        x.pop("id")
tab_chro.insert_many(all_chronicle)

tab_incidents.create_index(["rg_id"])
tab_src.create_index(["rg_id"])

# tab_incidents.create_index(["id"])

In [12]:
def add_state_country(row):
    #     print(row)
    chro = tab_chro.find_one(chronicler_name=row["chronicler_name"])
    #     print(chro)
    sub = pycountry.subdivisions.get(code=chro["iso3166_2"])
    assert sub is not None
    row["state"] = sub.name
    row["country"] = _(sub.country.name)
    return row

In [13]:
county_words = ["Landkreis", "Landkeis", "Kreis", "LK"]


def clean_county(x):
    if x is None or x == "None":
        return None
    x = clean_string(x)

    for w in county_words:
        w += " "
        if x.startswith(w):
            x = x[len(w) :]

    if not is_valid_county(x):
        print("removing", x)
        return None
    return x


def clean_city(x):
    x = clean_string(x)
    if x is None:
        return None
    assert len(x) > 0
    return x


def clean_string(x):
    x = clean(x, lang="de", lower=False)
    if len(x) == 0:
        return None
    return x

In [14]:
def fill_missing_county():
    statement = "SELECT * FROM incidents GROUP BY city, state having count(*) > 1"
    for row in db.query(statement):
        dupli = list(tab_incidents.find(city=row["city"], state=row["state"]))
        county_can = []
        contains_none = False
        for d in dupli:
            if d["county"] is not None:
                county_can.append(d["county"])
            else:
                contains_none = True

        unique_can = list(set(county_can))
        if contains_none and len(unique_can) == 1:
            tab_incidents.update(
                {"city": row["city"], "county": unique_can[0]}, ["city"]
            )
            print(unique_can)

In [15]:
# clean location text because there were still some errors
for x in tqdm(tab_incidents.all()):
    if x["rg_id"] in manual_fixes:
        x = {**x, **manual_fixes[x["rg_id"]]}

    x["county"] = x["orig_county"] = clean_string(x["county"])
    x["city"] = x["orig_city"] = clean_string(x["city"])

    if "address" in x and x["address"] is not None:
        x["address"] = x["orig_address"] = clean_string(x["address"])

    x = add_state_country(x)

    if x["date"] is None:
        print("date is broken, skipping")
        print(x)
        tab_incidents.delete(id=x["id"])
        continue
    #         raise ValueError

    # ignore older data
    if x["date"].year < 1990:
        tab_incidents.delete(id=x["id"])

    x["date"] = ensure_date(x["date"])

    #   manual fix
    if x["city"] == "Zerbst" and x["state"] == "Sachsen-Anhalt":
        x["city"] = "Zerbst/Anhalt"

    tab_incidents.update(x, ["id"])

622it [00:01, 495.72it/s]

date is broken, skipping
OrderedDict([('id', 564), ('chronicler_name', 'München Chronik'), ('tags', 'Der Dritte Weg'), ('motives', 'Gegen Geflüchtete, Rassismus'), ('contexts', 'Flugblattverteilaktion'), ('factums', ''), ('city', 'Landkreis München'), ('description', 'Weil alle Unterkünfte überfüllt sind, bringen die Landkreisbehörden vorübergehend Asylsuchende in der Turnhalle des Förderzentrums Unterschleißheim unter. Wie in einer vergleichbaren Situation im August 2014 schon einmal, verteilen daraufhin Aktivist_innen der Neonazipartei „Der Dritte Weg“ rassistische Flugblätter in die Briefkästen von Anwohner_innen.\n\n\nBei den verteilten Pam,phleten handelt es sich um die bekannten „Dritte-Weg“-Flugblätter „Asylmißbrauch in Deutschland endlich stoppen“. In den vom „Dritten-Weg“-Vorsitzenden Klaus Armstroff (Weidenthal) presserechtlich verantworteten Flugblättern heißt es u. a.: \n„(…) zahllose vollkommen kulturfremde Menschen werden mit den hart erarbeiteten Steuermitteln deutscher 

872it [00:01, 481.33it/s]

date is broken, skipping
OrderedDict([('id', 790), ('chronicler_name', 'ezra'), ('tags', None), ('motives', None), ('contexts', None), ('factums', None), ('city', 'Jena/Erfurt'), ('description', 'Während der Wahlkampfperiode wurden Morddrohungen gegen mehrere Politiker*innen ausgesprochen. Sie richten sich unter anderem gegen den CDU-Landeschef Mike Mohring und Dirk Adams, den Fraktionschef der Thüringer Grünen. Mohring erhielt Ende September erst eine handgeschriebene Postkarte mit Drohungen und Beleidigungen, in der indirekt Bezug auf den Mord am Kasseler Regierungspräsidenten Walter Lübcke genommen wurde. Im Oktober folgte dann eine Drohmail in der die Absender*innen einen Messer- oder Autobombenanschlag auf seine Person ankündigten, sollte Mohring nicht sofort seinen Wahlkampf einstellen. Dirk Adams erhielt einige Tage später eine E-Mail mit einer ähnlichen Drohung der „Cyber-Reichswehr“, wenn er nicht bei den Grünen austreten sollte.'), ('title', 'Morddrohungen gegen Politiker*inn

14466it [00:26, 563.53it/s]

...

14691it [00:26, 551.19it/s]

date is broken, skipping
OrderedDict([('id', 14587), ('chronicler_name', 'Opferberatung Rheinland'), ('tags', None), ('motives', None), ('contexts', None), ('factums', None), ('city', 'Grevenbroich'), ('description', 'Unbekannte Täter besprühten in der Nacht von Dienstag auf Mittwoch das Auto einer Grevenbroicher Familie mit rassistischen und rechten Parolen. Die Betroffenen und die Polizei gehen aufgrund der Tatumstände davon aus, dass es sich um eine gezielte Tat handelt und der oder die Täter die Familie kennen. Der Staatsschutz nahm Ermittlungen wegen einer fremdenfeindlich motivierten Straftat auf.'), ('title', None), ('date', None), ('rg_id', 'https://www.opferberatung-rheinland.de/chronik-der-gewalt/detail/5/6416-grevenbroich'), ('url', 'https://www.opferberatung-rheinland.de/chronik-der-gewalt/detail/5/6416-grevenbroich'), ('county', None), ('postal_code', None), ('state', 'Nordrhein-Westfalen'), ('latitude', None), ('longitude', None), ('address', None), ('orig_county', None),

14802it [00:27, 535.04it/s]

date is broken, skipping
OrderedDict([('id', 14698), ('chronicler_name', 'Opferberatung Rheinland'), ('tags', None), ('motives', None), ('contexts', None), ('factums', None), ('city', 'Dortmund'), ('description', 'Rechtsextreme griffen in Dortmund innerhalb von wenigen Tagen dreimal einen 26-jährigen Juden an. Am Sonntagmittag beschimpften und beleidigten ihn drei polizeibekannte Neonazis aus Dortmund und Schwelm, einen Faustschlag in Richtung seines Kopfes konnte der Betroffene zum Glück abwehren. Am Sonntagabend traf er erneut auf die Gewalttäter, die mehrfach den Hitlergruß zeigten und den Mann antisemitisch beleidigten und bedrohten. Drei Tage zuvor war er am Rande einer Neonazi-Kundgebung von dem 21-Jährigen Neonazi aus Schwelm antisemitisch beleidigt und gestoßen worden.'), ('title', None), ('date', None), ('rg_id', 'https://www.opferberatung-rheinland.de/chronik-der-gewalt/detail/21-u-24618-dortmund'), ('url', 'https://www.opferberatung-rheinland.de/chronik-der-gewalt/detail/21-

16042it [00:29, 542.88it/s]


In [17]:
# fill_missing_county()

In [18]:
def geocode_all():
    tab_incidents.create_column("district", db.types.text)
    statement = (
        "SELECT DISTINCT address, city, district, county, state, country FROM incidents"
    )
    subs = list(db.query(statement))

    # the geocoding api has problems with Leipzig as County. (There is a Landkreis Leipzig and a seperate City Leipzig)
    removed_counties_with_ids = []
    for i, x in enumerate(subs):
        if x["state"] is None:
            print(x)
        if x["county"] == "Leipzig":
            removed_county = x.pop("county")
            x["county"] = None
            removed_counties_with_ids.append([removed_county, i])

    r = requests.post(
        "https://geocode.app.vis.one/",
        auth=auth,
        json={"provider": "here", "locations": [{"query": dict(x)} for x in subs]},
    )
    r.raise_for_status()
    subs_location = r.json()["locations"]

    # add back county since it was correct
    for x, i in removed_counties_with_ids:
        subs_location[i]["query"]["county"] = x

    return subs_location

In [19]:
subs_location = geocode_all()

In [20]:
def geocode_second(subs_location):
    second_check = []
    second_check_ids = []

    for i, x in enumerate(subs_location):
        if len(x) == 1 and "county" in x["query"] and x["query"]["county"] is not None:
            x["query"]["city"] = x["query"]["city"] + ", " + x["query"]["county"]
            x["query"]["county"] = None
            second_check.append(x)
            second_check_ids.append(i)

    print(len(second_check))
    if len(second_check) == 0:
        return subs_location
    r = requests.post(
        "https://geocode.app.vis.one/",
        auth=auth,
        json={
            "provider": "here",
            "locations": [{"query": x["query"]} for x in second_check],
        },
    )
    r.raise_for_status()

    for i, x in enumerate(r.json()["locations"]):
        if len(x) != 1:
            subs_location[second_check_ids[i]] = x
            print("found!", x)
    return subs_location

In [21]:
# not used right now?
# subs_location = geocode_second(subs_location)

In [22]:
good_locations = []

for x in subs_location:
    if len(x) == 1:
        #         could not geocode these locations
        print("error here, deleting for now", x)
    #         tab_incidents.delete(**x['query'])
    else:
        #         rename
        query = x.pop("query")
        x["query_county"] = query["county"]
        x["query_city"] = query["city"]
        x["query_address"] = query["address"]
        good_locations.append(x)

error here, deleting for now {'query': {'address': None, 'city': 'Unbekannt', 'country': 'Deutschland', 'county': None, 'district': None, 'state': 'Bayern'}}
error here, deleting for now {'query': {'address': None, 'city': 'BA 02 Ludwigsvorstadt - Isarvorstadt, BA 04 Schwabing-West, BA 05 Au - Haidhausen, BA 06 Sendling, BA 09 Neuhausen - Nymphenburg, BA 23 Allach - Untermenzing', 'country': 'Deutschland', 'county': None, 'district': None, 'state': 'Bayern'}}
error here, deleting for now {'query': {'address': None, 'city': 'Heideberg (BaWü)', 'country': 'Deutschland', 'county': None, 'district': None, 'state': 'Rheinland-Pfalz'}}
error here, deleting for now {'query': {'address': None, 'city': None, 'country': 'Deutschland', 'county': None, 'district': None, 'state': 'Thüringen'}}
error here, deleting for now {'query': {'address': None, 'city': None, 'country': 'Deutschland', 'county': None, 'district': None, 'state': 'Hessen'}}
error here, deleting for now {'query': {'address': None, 

In [23]:
tab_loc = db["locations"]
tab_loc.insert_many(good_locations)

In [24]:
tab_loc.create_index(["id"])

## Merging Locations

We are trying out several ways to merge the location with geolocation back to the old without. Since we maniputlated the county etc., we have to try varioous ways how to merge. This should get improved (TDOO)


Not really sure whether a seperate table for location is needed. It was introduced because in some cases, multiple locations are associated with incident.

In [25]:
for x in tqdm(tab_incidents.all()):
    real_lat, real_long = x['latitude'], x['longitude']
    x_query = {name: x[name] for name in ["state", "country"]}
    row_loc = tab_loc.find_one(
        query_county=x["orig_county"],
        query_city=x["orig_city"],
        query_address=x["address"],
        **x_query
    )
    if row_loc is None:
        row_loc = tab_loc.find_one(
            query_county=x["county"],
            query_city=x["orig_city"],
            query_address=x["address"],
            **x_query
        )
        if row_loc is None:
            row_loc = tab_loc.find_one(
                query_county=x["orig_county"],
                query_city=x["city"],
                query_address=x["address"],
                **x_query
            )
            if row_loc is None:
                row_loc = tab_loc.find_one(
                    query_county=x["county"],
                    query_city=x["city"],
                    query_address=x["address"],
                    **x_query
                )
                if row_loc is None:
                    print(x)
                    print(x_query)
                    continue
    row_loc_geo = {
        name: row_loc[name]
        for name in [
            "latitude",
            "longitude",
            "postal_code",
            "street",
            "house_number",
            "district",
            "city",
            "county",
            "state",
            "country",
        ]
    }

    if real_lat is not None and real_long is not None:
        row_loc_geo['latitude'] = real_lat
        row_loc_geo['longitude'] = real_long
        print('added again!')
        print(x)

    merged = {**x, **row_loc_geo}
    tab_incidents.update(merged, ["id"])

0it [00:00, ?it/s]

OrderedDict([('id', 11), ('chronicler_name', 'München Chronik'), ('tags', ''), ('motives', ''), ('contexts', ''), ('factums', ''), ('city', 'Unbekannt'), ('description', 'Eine 71-Jährige mit Wohnsitz in München schreit am Sonntagnachmittag gegen 16.00 Uhr in der Maximilianstraße lautstark rassistische Parolen. Zeug*innen alarmieren daraufhin die Polizei, welche die Frau im Bereich des Karlsplatzes antreffen und festnehmen kann. Die 71-Jährige wird zur Behandlung in ein psychiatrisches Krankenhaus eingewiesen, da laut Polizei \n„aufgrund ihres Verhaltens und weiterer Äußerungen auch eine Eigengefährdung der Frau nicht ausgeschlossen werden konnte“\n.'), ('title', 'Rassistische Parolen'), ('date', datetime.datetime(2020, 11, 8, 0, 0)), ('rg_id', 'https://muenchen-chronik.de/8-november-2020-rassistische-parolen/'), ('url', 'https://muenchen-chronik.de/8-november-2020-rassistische-parolen/'), ('county', None), ('postal_code', None), ('state', 'Bayern'), ('latitude', None), ('longitude', No

76it [00:00, 358.53it/s]

OrderedDict([('id', 37), ('chronicler_name', 'München Chronik'), ('tags', ''), ('motives', 'Antisemitismus'), ('contexts', 'Öffentlicher Raum allgemein'), ('factums', 'Beleidigung/Beschimpfung/Bedrohung'), ('city', 'Unbekannt'), ('description', 'Die „Recherche- und Informationsstelle Antisemitismus Bayern“ (RIAS Bayern) berichtet auf Facebook im Nachhinein von einem antisemitischen Vorfall in der Stadt:\n\n\nEine Passantin lief mit einem Freund im Zentrum Münchens auf dem Gehsteig, als auf einmal eine Radfahrerin laut \n„Judensau“\n rief. Die Passantin war schockiert und dachte, ihr Begleiter, der Jude ist, sei gemeint. Dann bemerkte sie, dass sich die Radlerin im Streit mit einem weiteren Radfahrer befand und diesen beschimpfte. Die Zeugin informierte RIAS Bayern, da sie nicht möchte, dass der Vorfall untergeht.'), ('title', 'Antisemitische Beschimpfung'), ('date', datetime.datetime(2020, 9, 9, 0, 0)), ('rg_id', 'https://muenchen-chronik.de/9-september-2020-antisemitische-beschimpfung

214it [00:00, 415.94it/s]

...

6840it [00:14, 459.23it/s]

OrderedDict([('id', 6790), ('chronicler_name', 'Mobile Opferberatung'), ('tags', None), ('motives', None), ('contexts', None), ('factums', None), ('city', None), ('description', 'Zwei Männer beschimpfen gegen 19 Uhr in einer Straßenbahn an der Vogelweide einen 35-jährigen Mann aus Libyen. Als dieser am Marktplatz umsteigen will, schlagen die Unbekannten ihm gegen den Oberkörper. Dann versucht der 35-Jährige gleichzeitig mit den Angreifern in eine andere Bahn einzusteigen, wird aber von den beiden aus dem Fahrzeug gestoßen. Der Betroffene bleibt unverletzt. Der polizeiliche Staatsschutz ermittelt.'), ('title', None), ('date', datetime.datetime(2015, 9, 22, 0, 0)), ('rg_id', 'mobile-opferberatung-ca0dee4be1029c2ab24cdea755b88086'), ('url', 'http://www.mobile-opferberatung.de/monitoring/chronik2015/'), ('county', None), ('postal_code', None), ('state', 'Sachsen-Anhalt'), ('latitude', None), ('longitude', None), ('address', None), ('orig_county', None), ('orig_city', None), ('country', 'De

10256it [00:22, 454.34it/s]

OrderedDict([('id', 10208), ('chronicler_name', 'RAA Sachsen'), ('tags', None), ('motives', None), ('contexts', None), ('factums', None), ('city', 'Dresden, Dresdner Heide'), ('description', 'Auf einem Feld in der Dresdner Heide, in der Nähe des Hammerweges, haben Unbekannte aus mehreren faustgroßen Steinen ein Hakenkreuz gelegt. Dieses hatte eine Größe von 6,50m x 6,80m. Die Kriminalpolizei hat die weiteren Ermittlungen übernommen.'), ('title', 'Hakenkreuz aus Steinen gelegt'), ('date', datetime.datetime(2013, 12, 23, 0, 0)), ('rg_id', 'https://www.raa-sachsen.de/support/chronik/vorfaelle/dresdner-heide-3485'), ('url', 'https://www.raa-sachsen.de/support/chronik/vorfaelle/dresdner-heide-3485'), ('county', None), ('postal_code', None), ('state', 'Sachsen'), ('latitude', None), ('longitude', None), ('address', None), ('orig_county', None), ('orig_city', 'Dresden, Dresdner Heide'), ('country', 'Deutschland'), ('orig_address', None), ('district', None)])
{'state': 'Sachsen', 'country': 'D

15994it [00:34, 458.79it/s]


In [26]:
# we are using the final output locations of the incidents because we actually manupulated / discared some locations (if there already had a geolocation)
final_loc = list(
    tab_incidents.distinct(
        *[
            "latitude",
            "longitude",
            "postal_code",
            "street",
            "house_number",
            "district",
            "city",
            "county",
            "state",
            "country",
        ]
    )
)

In [27]:
len(final_loc)

2049

In [28]:
tab_loc.drop()

In [29]:
# tab_loc_final = db['locations_final']
tab_loc.insert_many(final_loc)

In [30]:
len(list(tab_loc.all()))

2049